# Tratamento dos Dados

In [2]:
import pandas as pd
from unidecode import unidecode

In [3]:
df = pd.read_csv('https://bi.static.es.gov.br/covid19/MICRODADOS.csv', sep = ';', encoding = 'cp1252')
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG
0,2020-11-05,2020-11-05,2020-11-04,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
1,2020-11-05,2020-11-05,2020-11-04,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
2,2020-11-05,2020-11-05,2020-11-05,NaN,2020-10-23,NaN,NaN,2020-11-05,NaN,Descartados,...,Não Informado,Não Informado,Não Informado,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado
3,2020-11-05,2020-11-05,2020-10-27,2020-11-03,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Sim,Não Informado,Não Informado,Não Informado,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
4,2020-11-05,2020-11-05,2020-11-02,2020-11-05,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não Informado,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado


In [4]:
print('Classificacão: {}'.format(df['Classificacao'].unique()))
print('Evolução: {}'.format(df['Evolucao'].unique()))

Classificacão: ['Suspeito' 'Descartados' 'Confirmados']
Evolução: ['-' 'Ignorado' 'Cura' 'Óbito pelo COVID-19' 'Óbito por outras causas']


In [5]:
total_casos = df.query('Classificacao == "Confirmados"').shape[0]
total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_curas = df.query('Evolucao == "Cura"').shape[0]

print('TOTAIS')
print('=================')
print('  - CASOS: {}'.format(total_casos))
print('  - ÓBITOS: {} ({}%)'.format(total_obitos, round((total_obitos * 100) / total_casos, 1)))
print('  - CURA: {} ({}%)'.format(total_curas, round((total_curas * 100) / total_casos, 1)))

TOTAIS
  - CASOS: 159813
  - ÓBITOS: 3896 (2.4%)
  - CURA: 149503 (93.5%)


In [6]:
df['DataNotificacao'] = pd.to_datetime(df['DataNotificacao'])
df.sort_values('DataNotificacao', inplace=True)
df = df.query('Classificacao == "Confirmados"').reset_index(drop=True)

## Padroniza nome de municípios e bairros

In [7]:
df['Municipio'] = df['Municipio'].apply(lambda x: unidecode(str(x)).upper())
df['Bairro'] = df['Bairro'].apply(lambda x: unidecode(str(x)).upper())

## Cálculo de Confirmados

In [8]:
df['ConfirmadosAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumcount() + 1

df['Confirmados'] = df[['Municipio', 'Bairro', 'DataNotificacao']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumcount() + 1

## Cálculo de Óbitos

In [9]:
df['Obitos'] = df['Evolucao'].apply(lambda evolucao: 1 if evolucao == 'Óbito pelo COVID-19' else 0)

df['ObitosAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df['Obitos'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumsum()

## Cálculo de Curas

In [10]:
df['Curas'] = df['Evolucao'].apply(lambda evolucao: 1 if evolucao == 'Cura' else 0)

df['CurasAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df['Curas'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumsum()

## Contagem de Casos, Óbitos e Curas

In [11]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = df[['DataNotificacao']].drop_duplicates().reset_index(drop=True)
municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)

datas['key'] = 0
municipios_bairros['key'] = 0

In [12]:
df_counts = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_counts.merge(
    df[[
        'Municipio',
        'Bairro',
        'DataNotificacao',
        'Confirmados',
        'Obitos',
        'Curas',
        'ConfirmadosAcumulado',
        'ObitosAcumulado',
        'CurasAcumulado'
    ]].drop_duplicates(grupo_base, keep='last')\
        .dropna(),
    on=grupo_base,
    how='left'
)

df_counts.fillna(
    {
        'Confirmados': 0,
        'Obitos': 0,
        'Curas': 0,
    },
    inplace=True
)

columns_ffill = ['ConfirmadosAcumulado', 'ObitosAcumulado', 'CurasAcumulado']

df_counts[columns_ffill] = df_counts.groupby(['Municipio', 'Bairro'])\
    .fillna(method='ffill')\
    .fillna(0)\
    [columns_ffill]

In [13]:
df_counts.to_csv('../notebooks_output/microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')